In [1]:
import numpy as np
from metabci.brainda.datasets import Wang2016, BETA
from metabci.brainda.paradigms import SSVEP
from metabci.brainda.algorithms.decomposition import FBSCCA,SCCA
from metabci.brainda.algorithms.decomposition.base import generate_filterbank, generate_cca_references

In [2]:
dataset = Wang2016()

events = dataset.events.keys()
freq_list = [dataset.get_freq(event) for event in events]

paradigm = SSVEP(
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],
    intervals=[(0.14, 1.14)],
    srate=250
)

# add 5-90Hz bandpass filter in raw hook
def raw_hook(raw, caches):
    # do something with raw object
    raw.filter(5, 90, l_trans_bandwidth=2,h_trans_bandwidth=5,
        phase='zero-double')
    caches['raw_stage'] = caches.get('raw_stage', -1) + 1
    return raw, caches

paradigm.register_raw_hook(raw_hook)


subjects = [22,24,27,28,31,34,35] #  用于测试的受试者ID
X, y, meta = paradigm.get_data(
    dataset,
    subjects=subjects,
    return_concat=True,
    n_jobs=None,
    verbose=False)

--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z


In [3]:
Yf = generate_cca_references(freq_list, srate=250, T= 1, n_harmonics = 3)

wp=[(6,90),(14,90),(22,90),(30,90),(38,90)]
ws=[(4,100),(10,100),(16,100),(24,100),(32,100)]
filterbank = generate_filterbank(wp,ws,srate=250,order=15,rp=1)
filterweights = [(idx_filter+1) ** (-1.25) + 0.25 for idx_filter in range(5)]
estimator = FBSCCA(filterbank=filterbank, n_components=1, filterweights=np.array(filterweights), n_jobs=-1)
#estimator = SCCA(n_components=1, n_jobs=-1)
subjects = [22,24,27,28,31,34,35]
accs = []

for subj in subjects:
    # 划分训练 / 测试
    test_ind = np.where(meta['subject'] == subj)[0]
    # 拟合 + 预测
    p_labels = estimator.fit(X=X[test_ind], y=y[test_ind], Yf=Yf).predict(X[test_ind])
    # 计算该受试者的准确率
    acc = np.mean(p_labels == y[test_ind])
    accs.append(acc)
    print(f"Subject {subj}: {acc:.4f}")

# 平均准确率 
print("Mean accuracy:", np.mean(accs))

Subject 22: 0.8292
Subject 24: 0.6750
Subject 27: 0.6792
Subject 28: 0.6875
Subject 31: 0.4583
Subject 34: 0.5750
Subject 35: 0.3375
Mean accuracy: 0.605952380952381


In [2]:
BETA_dataset = BETA()
events = BETA_dataset.events.keys()
freq_list = [BETA_dataset.get_freq(event) for event in events]
#print(freq_list) # 输出频率显示

paradigm_3s = SSVEP(
    # In order to validate the data quality by visual inspection,
    # nine parietal and occipital channels(Pz, PO3, PO5, PO4, PO6, POz, O1, Oz,and O2) were selected。
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-3.5s ,每个试次包含事件开始前0.5秒的数据，以及事件时间窗后0.5秒的数据。对于S16–S70的被试，时间窗为 3秒，试次总长度为 4秒
    srate=250  # 采样率
)

# BETA数据集，已经通过matlab的eegfit，进行了3-90HZ的带通滤波，故此处不再进行滤波处理
# BETA_wof数据集没有进行滤波处理，链接已经
# 在tsinghua.py把BETA_URL做以下修改即可使用未滤波的版本
# BETA_URL = "https://bci.med.tsinghua.edu.cn/upload/liubingchuan_BETA_wof/"


# 数据存储为一个四维张量 [channel, time point, block, condition]
#                    [   64,       750,      4,      40    ]

# 获取训练数据
X_3s, y_3s, meta_3s = paradigm_3s.get_data(
    BETA_dataset,
    subjects=[36,37,49,62,66,67,70],
    return_concat=True,
    n_jobs=None,
    verbose=False
)

# 选择使用的BETA数据
# 2s和3s的数据shape不同，不能直接拼接，是否填0补充相同大小拼接？
X_BETA = X_3s
y_BETA = y_3s
meta_BETA = meta_3s
#print(X_BETA.shape)
#print(meta_BETA)


--------ssssss, /upload/liubingchuan/S31-S40.tar.gz
--------ssssss, /upload/liubingchuan/S31-S40.tar.gz
--------ssssss, /upload/liubingchuan/S41-S50.tar.gz
--------ssssss, /upload/liubingchuan/S61-S70.tar.gz
--------ssssss, /upload/liubingchuan/S61-S70.tar.gz
--------ssssss, /upload/liubingchuan/S61-S70.tar.gz
--------ssssss, /upload/liubingchuan/S61-S70.tar.gz


In [3]:
Yf = generate_cca_references(freq_list, srate=250, T= 1, n_harmonics = 3)

wp=[(6,90),(14,90),(22,90),(30,90),(38,90)]
ws=[(4,100),(10,100),(16,100),(24,100),(32,100)]
filterbank = generate_filterbank(wp,ws,srate=250,order=15,rp=1)
filterweights = [(idx_filter+1) ** (-1.25) + 0.25 for idx_filter in range(5)]
estimator = FBSCCA(filterbank=filterbank, n_components=1, filterweights=np.array(filterweights), n_jobs=-1)
#estimator = SCCA(n_components=1, n_jobs=-1)
subjects = [36,37,49,62,66,67,70]
accs = []

for subj in subjects:
    # 划分训练 / 测试
    test_ind = np.where(meta_BETA['subject'] == subj)[0]
    # 拟合 + 预测
    p_labels = estimator.fit(X=X_BETA[test_ind], y=y_BETA[test_ind], Yf=Yf).predict(X_BETA[test_ind])
    # 计算该受试者的准确率
    acc = np.mean(p_labels == y_BETA[test_ind])
    accs.append(acc)
    print(f"Subject {subj}: {acc:.4f}")

# 平均准确率 
print("Mean accuracy:", np.mean(accs))

Subject 36: 0.4813
Subject 37: 0.4875
Subject 49: 0.7250
Subject 62: 0.3625
Subject 66: 0.5563
Subject 67: 0.6625
Subject 70: 0.4938
Mean accuracy: 0.5383928571428571
